In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

x = 10

## Summary Statistics function

In [6]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

def summary_statistics_g(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'saves': 'sum',
        'shots': 'sum',
        'start': 'sum',
        'shutout': 'sum',
        'fantasyPoints': 'sum'
    }).reset_index()
    
    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)
    
    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

# Pull in the data files from scrape

In [10]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# file_name = f"data/manual-adjustments.xlsx"
rankings = pd.read_excel(file_name, index_col=False)

# fileName = f"data/summary_stats_last{x}-{today.strftime('%Y-%m-%d')}.csv"
# last_10 = pd.read_csv(fileName, index_col=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/allG_df_fp-2023-12-05.csv'

In [48]:
ordercolumns = rankings.columns.to_list()
ordercolumns

['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-11-27',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01']

## Code to re-tag rankings for save

In [49]:
# file_name = f"data/manual-adjustments.xlsx"
# manual = pd.read_excel(file_name)
# for index, row in rankings.iterrows():
#     try:
#         rankings.at[index, 'rank_2023-11-27'] = player_bios_adjusted.loc[player_bios_adjusted['espnId'] == row['espnId'], 'rank'].iloc[0]
#     except:
#         rankings.at[index, 'rank_2023-11-27'] = 800

In [50]:
# fileName = f"data/rankings_worksheet_2023-11-27.xlsx"
# rankings.to_excel(fileName, index=False)

## New column order

In [51]:
column_order = ['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-12-04',                
 'rank_2023-11-27',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01'
]

In [52]:
player_bios

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId
0,8481559,J. Hughes,NJD,C,21649,17,57.6,9.58,3.39,0,60,203.40,Jack Hughes,2001-05-14,L,71,175,99.711120,C,4565222
1,8476453,N. Kucherov,TBL,R,31088,24,82.5,9.55,3.44,0,57,196.08,Nikita Kucherov,1993-06-17,L,71,182,99.874337,RW,2563060
2,8479318,A. Matthews,TOR,C,28457,22,71.4,9.03,3.25,0,60,195.00,Auston Matthews,1997-09-17,L,75,215,99.916225,C,4024123
3,8480069,C. Makar,COL,D,33757,23,75.9,8.09,3.30,0,59,194.70,Cale Makar,1998-10-30,R,71,187,99.906106,D,4233563
4,8475660,C. Talbot,LAK,G,53370,15,61.8,4.17,4.12,0,45,190.75,Cam Talbot,1987-07-05,L,76,200,82.242923,G,5734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,8470594,M. Fleury,MIN,G,32280,9,-4.6,-0.51,-0.51,0,25,-13.29,Marc-Andre Fleury,1984-11-28,L,74,185,14.589481,G,2346
826,8476883,A. Vasilevskiy,TBL,G,13040,4,-7.0,-1.93,-1.75,0,8,-16.04,Andrei Vasilevskiy,1994-07-25,L,76,220,93.859839,G,2976847
827,8477480,E. Comrie,BUF,G,17144,6,-9.6,-2.02,-1.60,0,11,-23.35,Eric Comrie,1995-07-06,L,73,183,0.271932,G,3042057
828,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,12,-23.50,Jack Campbell,1992-01-09,L,75,200,2.425794,G,5473


# Adjusted Fantasy Remain, done with Last 10

In [53]:
for index, row in player_bios.iterrows():
    if row['position'] != 'G':
        player_log = all_df.loc[all_df['playerId'] == row['playerId']]
        player_log.sort_values('gameDate', inplace=True, ascending=False)
        player_log = player_log.head(10)
        last_10_sum = summary_statistics(player_log)
        try:
            last_10_FPPG = last_10_sum['FPPG'].iloc[0]
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'last10FP'] = last_10_sum['fantasyPoints'].iloc[0]
            player_bios.at[index, 'adjFantasyRemain'] = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
        except:
            player_bios.at[index, 'last10FPPG'] = 0
            player_bios.at[index, 'last10FP'] = 0
            player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
    else:
        player_log = allG_df.loc[allG_df['playerId'] == row['playerId']]
        player_log.sort_values('gameDate', inplace=True, ascending=False)
        player_log = player_log.head(10)
        last_10_sum = summary_statistics_g(player_log)
        try:
            last_10_FPPG = last_10_sum['FPP60'].iloc[0]
            player_bios.at[index, 'last10FP'] = last_10_sum['fantasyPoints'].iloc[0]
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'adjFantasyRemain'] = (((row['FPP60'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
        except:
            player_bios.at[index, 'last10FPPG'] = 0
            player_bios.at[index, 'last10FP'] = 0
            player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']

player_bios.sort_values('adjFantasyRemain')

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,last10FP,adjFantasyRemain
829,8482821,A. Soderblom,CHI,G,35889,11,-10.0,-1.00,-0.91,0,...,1999-08-19,L,75,180,0.849100,G,4894729,-1.34,-12.0,-30.06
828,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,1992-01-09,L,75,200,2.425794,G,5473,-1.93,-8.6,-23.16
827,8477480,E. Comrie,BUF,G,17144,6,-9.6,-2.02,-1.60,0,...,1995-07-06,L,73,183,0.271932,G,3042057,-2.02,-9.6,-22.22
826,8476883,A. Vasilevskiy,TBL,G,13040,4,-7.0,-1.93,-1.75,0,...,1994-07-25,L,76,220,93.859839,G,2976847,-1.93,-7.0,-15.44
825,8470594,M. Fleury,MIN,G,32280,9,-4.6,-0.51,-0.51,0,...,1984-11-28,L,74,185,14.589481,G,2346,-0.51,-4.6,-12.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,8481559,J. Hughes,NJD,C,21649,17,57.6,9.58,3.39,0,...,2001-05-14,L,71,175,99.711120,C,4565222,2.54,25.4,186.40
8,8478402,C. McDavid,EDM,C,26057,20,58.9,8.14,2.94,0,...,1997-01-13,L,73,194,99.880110,C,3895074,3.71,37.1,191.80
1,8476453,N. Kucherov,TBL,R,31088,24,82.5,9.55,3.44,0,...,1993-06-17,L,71,182,99.874337,RW,2563060,3.32,33.2,193.80
4,8475660,C. Talbot,LAK,G,53370,15,61.8,4.17,4.12,0,...,1987-07-05,L,76,200,82.242923,G,5734,5.07,50.8,201.15


In [54]:
player_bios.sort_values('adjFantasyRemain', ascending=False).head(50)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,last10FP,adjFantasyRemain
3,8480069,C. Makar,COL,D,33757,23,75.9,8.09,3.30,0,...,1998-10-30,R,71,187,99.906106,D,4233563,3.69,36.9,202.370000
4,8475660,C. Talbot,LAK,G,53370,15,61.8,4.17,4.12,0,...,1987-07-05,L,76,200,82.242923,G,5734,5.07,50.8,201.150000
1,8476453,N. Kucherov,TBL,R,31088,24,82.5,9.55,3.44,0,...,1993-06-17,L,71,182,99.874337,RW,2563060,3.32,33.2,193.800000
8,8478402,C. McDavid,EDM,C,26057,20,58.9,8.14,2.94,0,...,1997-01-13,L,73,194,99.880110,C,3895074,3.71,37.1,191.800000
0,8481559,J. Hughes,NJD,C,21649,17,57.6,9.58,3.39,0,...,2001-05-14,L,71,175,99.711120,C,4565222,2.54,25.4,186.400000
5,8477956,D. Pastrnak,BOS,R,26861,23,73.1,9.80,3.18,0,...,1996-05-25,R,72,196,99.906114,RW,3114778,2.96,29.6,183.293333
7,8480865,N. Dobson,NYI,D,34704,23,68.7,7.13,2.99,0,...,2000-01-07,R,76,200,95.598729,D,4352732,3.02,30.2,177.000000
2,8479318,A. Matthews,TOR,C,28457,22,71.4,9.03,3.25,0,...,1997-09-17,L,75,215,99.916225,C,4024123,2.33,23.3,176.600000
10,8477492,N. MacKinnon,COL,C,30923,23,67.3,7.83,2.93,0,...,1995-09-01,R,72,200,99.898888,C,3041969,3.06,30.6,175.426667
9,8483495,S. Nemec,NJD,D,1358,1,2.9,7.69,2.90,0,...,2004-02-15,R,73,190,4.512495,D,4915344,2.90,2.9,174.000000


# Get adjustment to Fantasy Remain

In [55]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [56]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)
opp_defense

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,165735,60,16,214,122,205,239.1,24,22,9.96,7,Anaheim Ducks
1,ARI,160180,40,15,215,169,195,212.4,23,11,9.23,13,Arizona Coyotes
2,BOS,159318,28,6,198,180,221,193.3,23,9,8.40,27,Boston Bruins
3,BUF,167476,42,14,187,141,217,217.3,24,10,9.05,16,Buffalo Sabres
4,CAR,159645,33,9,141,137,241,205.3,23,10,8.93,20,Carolina Hurricanes
5,CBJ,175399,47,12,235,115,275,248.0,25,8,9.92,8,Columbus Blue Jackets
6,CGY,169519,44,14,190,151,260,241.1,24,8,10.05,5,Calgary Flames
7,CHI,152688,47,12,194,127,203,212.1,22,12,9.64,11,Chicago Blackhawks
8,COL,158740,33,6,170,157,228,196.7,23,8,8.55,24,Colorado Avalanche
9,DAL,152958,31,12,202,105,214,197.2,22,6,8.96,19,Dallas Stars


In [57]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,86560,63,704,62.2,24,520,564,0,95,112,2.59,10,Anaheim Ducks
1,ARI,82966,71,622,18.2,23,452,502,1,89,109,0.79,26,Arizona Coyotes
2,BOS,82169,73,731,16.6,23,523,580,0,109,125,0.72,27,Boston Bruins
3,BUF,86079,65,686,54.2,24,508,562,1,100,110,2.26,12,Buffalo Sabres
4,CAR,83283,77,783,30.2,23,591,646,1,91,108,1.31,23,Carolina Hurricanes
5,CBJ,89856,70,751,71.2,25,554,614,2,110,120,2.85,7,Columbus Blue Jackets
6,CGY,87147,65,759,70.8,24,582,634,1,95,105,2.95,5,Calgary Flames
7,CHI,78589,52,602,74.0,22,462,507,2,66,73,3.36,2,Chicago Blackhawks
8,COL,82689,77,736,19.8,23,518,571,3,122,141,0.86,25,Colorado Avalanche
9,DAL,79252,74,653,5.8,22,473,528,1,85,100,0.26,28,Dallas Stars


In [58]:
remainSked = homeOnlySked.loc[homeOnlySked['gameDT'] >= str(datetime.utcnow())]
remainSked.sort_values('gameDate', inplace=True)
remainSked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
92,2023020371,2023-12-03,Sunday 07:00 PM,CBJ,BOS,2023-12-04 00:00:00,0
50,2023020370,2023-12-03,Sunday 06:00 PM,SJS,NYR,2023-12-03 23:00:00,0
1199,2023020369,2023-12-03,Sunday 02:00 PM,CHI,MIN,2023-12-03 19:00:00,0
953,2023020372,2023-12-03,Sunday 07:00 PM,NSH,BUF,2023-12-04 00:00:00,0
298,2023020373,2023-12-03,Sunday 08:00 PM,COL,LAK,2023-12-04 01:00:00,0
...,...,...,...,...,...,...,...
1229,2023021307,2024-04-18,Thursday 07:00 PM,SEA,MIN,2024-04-18 23:00:00,0
40,2023021311,2024-04-18,Thursday 10:00 PM,ANA,VGK,2024-04-19 02:00:00,0
327,2023021312,2024-04-18,Thursday 10:30 PM,CHI,LAK,2024-04-19 02:30:00,0
491,2023021308,2024-04-18,Thursday 08:00 PM,VAN,WPG,2024-04-19 00:00:00,0


In [73]:
team_names_FPAPG = team_names.copy()
team_names_FPAPG['G_FPAPG'] = 0
team_names_FPAPG['F_FPAPG'] = 0
team_names_FPAPG['D_FPAPG'] = 0

for index, row in remainSked.iterrows():
    index_of_away = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['awayTeam']].index[0]
    index_of_home = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['homeTeam']].index[0]
    
    tempAwayG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'G_FPAPG'] += tempAwayG
    team_names_FPAPG.at[index_of_home, 'G_FPAPG'] += tempHomeG
    
    tempAwayF = opp_forward.loc[opp_forward['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeF = opp_forward.loc[opp_forward['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'F_FPAPG'] += tempAwayF
    team_names_FPAPG.at[index_of_home, 'F_FPAPG'] += tempHomeF
    
    tempAwayD = opp_defense.loc[opp_defense['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeD = opp_defense.loc[opp_defense['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'D_FPAPG'] += tempAwayD
    team_names_FPAPG.at[index_of_home, 'D_FPAPG'] += tempHomeD
    
# Gmean = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Fmean = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Dmean = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Gstd = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Fstd = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Dstd = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).std()

# for index, row in team_names_FPAPG.iterrows():
#     Gmodifier = (row['G_FPAPG']/row['gamesRemaining']) - Gmean
#     adjModifier = (Gmodifier - Gmean) / Gstd
    
#     team_names_FPAPG.at[index, 'Gsos'] = adjModifier
    
#     Fmodifier = (row['F_FPAPG']/row['gamesRemaining']) - Fmean
#     adjModifier = (Fmodifier - Fmean) / Fstd
    
#     team_names_FPAPG.at[index, 'Fsos'] = adjModifier
    
#     Dmodifier = (row['D_FPAPG']/row['gamesRemaining']) - Dmean
#     adjModifier = (Dmodifier - Dmean) / Dstd
    
#     team_names_FPAPG.at[index, 'Dsos'] = adjModifier
    
team_names_FPAPG['G_FPAPG_PG'] = team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['F_FPAPG_PG'] = team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['D_FPAPG_PG'] = team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['Gmod'] = team_names_FPAPG['G_FPAPG_PG'] / team_names_FPAPG['G_FPAPG_PG'].mean()
team_names_FPAPG['Fmod'] = team_names_FPAPG['F_FPAPG_PG'] / team_names_FPAPG['F_FPAPG_PG'].mean()
team_names_FPAPG['Dmod'] = team_names_FPAPG['D_FPAPG_PG'] / team_names_FPAPG['D_FPAPG_PG'].mean()
# team_names_FPAPG['GmodNorm'] = (team_names_FPAPG['Gmod'] - team_names_FPAPG['Gmod'].min()) / (team_names_FPAPG['Gmod'].max() - team_names_FPAPG['Gmod'].min())

team_names_FPAPG

,abbreviation,name,gamesPlayed,gamesRemaining,goalsFor,goalsAgainst,logo,conference,division,G_FPAPG,F_FPAPG,D_FPAPG,G_FPAPG_PG,F_FPAPG_PG,D_FPAPG_PG,Gmod,Fmod,Dmod
0,VGK,Vegas Golden Knights,25,57,82,59,https://assets.nhle.com/logos/nhl/svg/VGK_ligh...,Western,Pacific,101.08,923.91,532.63,1.773333,16.208947,9.344386,0.967664,1.006612,1.020741
1,NYR,New York Rangers,22,60,74,55,https://assets.nhle.com/logos/nhl/svg/NYR_ligh...,Eastern,Metropolitan,121.94,980.73,543.03,2.032333,16.345500,9.050500,1.108994,1.015093,0.988638
2,BOS,Boston Bruins,23,59,78,58,https://assets.nhle.com/logos/nhl/svg/BOS_ligh...,Eastern,Atlantic,108.01,955.06,536.58,1.830678,16.187458,9.094576,0.998956,1.005278,0.993453
3,VAN,Vancouver Canucks,25,57,96,64,https://assets.nhle.com/logos/nhl/svg/VAN_ligh...,Western,Pacific,101.61,905.56,519.44,1.782632,15.887018,9.112982,0.972738,0.986620,0.995463
4,COL,Colorado Avalanche,23,59,86,67,https://assets.nhle.com/logos/nhl/svg/COL_ligh...,Western,Central,105.93,943.74,539.62,1.795424,15.995593,9.146102,0.979718,0.993363,0.999081
5,DAL,Dallas Stars,22,60,79,62,https://assets.nhle.com/logos/nhl/svg/DAL_ligh...,Western,Central,114.82,976.44,552.55,1.913667,16.274000,9.209167,1.044240,1.010652,1.005970
6,FLA,Florida Panthers,24,58,72,62,https://assets.nhle.com/logos/nhl/svg/FLA_ligh...,Eastern,Atlantic,97.19,930.26,521.00,1.675690,16.038966,8.982759,0.914382,0.996056,0.981238
7,LAK,Los Angeles Kings,20,62,77,49,https://assets.nhle.com/logos/nhl/svg/LAK_ligh...,Western,Pacific,122.47,1011.77,582.62,1.975323,16.318871,9.397097,1.077885,1.013439,1.026499
8,DET,Detroit Red Wings,23,59,86,68,https://assets.nhle.com/logos/nhl/svg/DET_ligh...,Eastern,Atlantic,113.52,953.64,535.55,1.924068,16.163390,9.077119,1.049916,1.003783,0.991546
9,CAR,Carolina Hurricanes,23,59,81,75,https://assets.nhle.com/logos/nhl/svg/CAR_ligh...,Eastern,Metropolitan,102.76,938.81,534.09,1.741695,15.912034,9.052373,0.950400,0.988173,0.988842


# Adjusted remain based on sked

In [78]:
# for index, row in player_bios.iterrows():
#     if row['position'] == 'G':
#         player_log = allG_df.loc[allG_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics_g(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPPG'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Gmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
#     elif row['position'] == 'D':
#         player_log = all_df.loc[all_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPP60'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Dmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
#     else:
#         player_log = all_df.loc[all_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPP60'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Fmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']

# player_bios.sort_values('adjFantasyRemain')

for index, row in player_bios.iterrows():
    sosG = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Gmod'].iloc[0]
    sosF = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Fmod'].iloc[0]
    sosD = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Dmod'].iloc[0]
    if row['position'] == 'D':
        player_bios.at[index, 'adjSkedFantasyRemain'] = row['adjFantasyRemain'] * sosD
    elif row['position'] == 'G':
        player_bios.at[index, 'adjSkedFantasyRemain'] = row['adjFantasyRemain'] * sosG
    else:
        player_bios.at[index, 'adjSkedFantasyRemain'] = row['adjFantasyRemain'] * sosF
        
player_bios['adjFantasyRemain'] = round(player_bios['adjFantasyRemain'], 2)
player_bios['adjSkedFantasyRemain'] = round(player_bios['adjSkedFantasyRemain'], 2)

player_bios.sort_values('adjSkedFantasyRemain', ascending=False, inplace=True)
player_bios

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos,adjSkedFantasyRemain
4,8475660,C. Talbot,LAK,G,53370,15,61.8,4.17,4.12,0,...,5734,5.07,50.8,201.15,1,5.0,20.0,5.0,1.0,216.82
3,8480069,C. Makar,COL,D,33757,23,75.9,8.09,3.30,0,...,4233563,3.69,36.9,202.37,10,4.0,5.0,4.0,1.0,202.18
1,8476453,N. Kucherov,TBL,R,31088,24,82.5,9.55,3.44,0,...,2563060,3.32,33.2,193.80,13,2.0,3.0,2.0,1.0,192.09
8,8478402,C. McDavid,EDM,C,26057,20,58.9,8.14,2.94,0,...,3895074,3.71,37.1,191.80,9,9.0,4.0,9.0,4.0,190.46
0,8481559,J. Hughes,NJD,C,21649,17,57.6,9.58,3.39,0,...,4565222,2.54,25.4,186.40,44,1.0,6.0,1.0,1.0,185.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,8470594,M. Fleury,MIN,G,32280,9,-4.6,-0.51,-0.51,0,...,2346,-0.51,-4.6,-12.75,822,826.0,818.0,1042.0,73.0,-13.13
826,8476883,A. Vasilevskiy,TBL,G,13040,4,-7.0,-1.93,-1.75,0,...,2976847,-1.93,-7.0,-15.44,826,827.0,800.0,1043.0,74.0,-15.61
827,8477480,E. Comrie,BUF,G,17144,6,-9.6,-2.02,-1.60,0,...,3042057,-2.02,-9.6,-22.22,828,828.0,814.0,1044.0,75.0,-21.61
828,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,5473,-1.93,-8.6,-23.16,827,829.0,820.0,1045.0,76.0,-22.92


## Add new ranking

In [80]:
for index, row in rankings.iterrows():
    try:
        rankings.at[index,'fantasyRemain'] = player_bios.loc[player_bios['espnId'] == row['espnId']]['adjSkedFantasyRemain'].iloc[0]
    except:
        rankings.at[index,'fantasyRemain'] = 0


rankings['rank_2023-12-04'] = rankings['fantasyRemain'].rank(ascending=False, method='min').astype(int)
rankings 

rankings = rankings[column_order]
rankings.sort_values('rank_2023-12-04')

fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Benoit-Olivier Groulx': 'Bo Groulx',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
#     'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker',
    'Alex Barre-Boulet': 'Alex Barré-Boulet'
}

inverse_fixes = {value: key for key, value in fixes.items()}

rankings['FullName'].replace(fixes, inplace=True)
rankings.sort_values('rank_2023-12-04')

,espnId,FullName,fantasyRemain,rank_2023-12-04,rank_2023-11-27,rank_2023-11-20,rank_2023-11-13,rank_2023-11-06,rank_2023-10-30,rank_2023-10-23,rank_2023-09-01
46,5734,Cam Talbot,216.82,1,20,31.0,47.0,68.0,238.0,247.0,238.0
10,4233563,Cale Makar,202.18,2,5,6.0,11.0,11.0,9.0,9.0,17.0
5,2563060,Nikita Kucherov,192.09,3,3,5.0,6.0,9.0,10.0,13.0,26.0
2,3895074,Connor McDavid,190.46,4,4,2.0,3.0,2.0,1.0,1.0,1.0
4,4565222,Jack Hughes,185.30,5,6,4.0,5.0,5.0,5.0,7.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
641,4894487,Devon Levi,-11.67,1041,816,791.0,631.0,743.0,317.0,260.0,317.0
614,2346,Marc-Andre Fleury,-13.13,1042,818,802.0,602.0,701.0,349.0,260.0,349.0
82,2976847,Andrei Vasilevskiy,-15.61,1043,800,800.0,83.0,111.0,123.0,123.0,52.0
656,3042057,Eric Comrie,-21.61,1044,814,779.0,647.0,714.0,791.0,260.0,791.0


In [82]:
file_name = f"data/rankings_worksheet_{(today).strftime('%Y-%m-%d')}.xlsx"
# file_name = f"data/manual-adjustments.xlsx"
rankings.to_excel(file_name, index=False)

In [83]:
# last_10['rank'] = last_10['fantasyPoints'].rank(method='first', ascending=False)
# last_10['rank'] = last_10['rank'].astype(int)
# last_10.head(40)
player_bios['last10rank'] = player_bios['last10FP'].rank(ascending=False).astype(int)


In [84]:
player_bios
remain_rank = 0
for index, row in player_bios.iterrows():
    try:
        rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-12-04'].iloc[0]
        last_rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-11-27'].iloc[0]
#         past_10 = last_10.loc[rankings['espnId'] == row['espnId']]['rank'].iloc[0]
    except:
        rank = 300
        last_rank = 300
#         past_10 = 800
        
    remain_rank += 1
        
    player_bios.at[index, 'remain_rank'] =  remain_rank
    player_bios.at[index, 'last_rank'] =  last_rank
    player_bios.at[index, 'rank'] =  rank
#     player_bios.at[index, 'past_10'] =  past_10
    
not_ranked = player_bios.loc[player_bios['rank'] == 300]
for index, row in not_ranked.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")
    if row['remain_rank'] > 300:
        break

print('-------------')        

poor_rank = player_bios.loc[(player_bios['rank'] >= 250) & (player_bios['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")


C. Ingram: 14.0
H. Thrun: 33.0
N. Seeler: 105.0
D. Chisholm: 106.0
J. Bernard-Docker: 141.0
B. Biro: 184.0
M. Carcone: 192.0
N. Okhotiuk: 195.0
G. Lindström: 231.0
K. Korczak: 233.0
D. Mermis: 251.0
P. Wotherspoon: 255.0
L. Belpedio: 261.0
B. Faber: 275.0
B. Malenstyn: 284.0
J. Woll: 288.0
D. Samberg: 290.0
R. Poehling: 303.0
-------------
M. Necas: 254.0
N. Pionk: 258.0
A. Ekblad: 259.0
B. Pesce: 265.0
L. Hughes: 268.0
C. Atkinson: 269.0
T. Brodie: 271.0
F. Zetterlund: 273.0
P. Laine: 276.0
M. Appleton: 278.0
D. Strome: 282.0
V. Tarasenko: 287.0
J. Kotkaniemi: 289.0
P. Mintyukov: 291.0
D. Fabbro: 294.0
J. Farabee: 298.0
A. Carrier: 300.0
J. Binnington: 331.0
J. van Riemsdyk: 340.0
P. Dubois: 343.0
M. Boldy: 346.0
N. Paul: 357.0
R. Strome: 390.0
S. Monahan: 415.0
J. Spurgeon: 418.0
A. Kuzmenko: 436.0
A. Iafallo: 441.0
J. Schwartz: 459.0
T. Terry: 487.0
R. Smith: 500.0
B. Montour: 507.0
C. Hart: 567.0
V. Husso: 644.0
J. Saros: 725.0
J. Allen: 788.0
F. Gustavsson: 794.0


In [16]:
def convert_to_slug(name):
    # Convert to lowercase and replace spaces with hyphens
    slug = name.lower().replace(' ', '-')
    return slug

In [87]:
player_bios['rank_by_pos'] = player_bios.groupby('default_pos')['rank'].rank(ascending=True)
player_bios.head(50)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos,adjSkedFantasyRemain
4,8475660,C. Talbot,LAK,G,53370,15,61.8,4.17,4.12,0,...,5734,5.07,50.8,201.15,1,1.0,20.0,1.0,1.0,216.82
3,8480069,C. Makar,COL,D,33757,23,75.9,8.09,3.30,0,...,4233563,3.69,36.9,202.37,10,2.0,5.0,2.0,1.0,202.18
1,8476453,N. Kucherov,TBL,R,31088,24,82.5,9.55,3.44,0,...,2563060,3.32,33.2,193.80,13,3.0,3.0,3.0,1.0,192.09
8,8478402,C. McDavid,EDM,C,26057,20,58.9,8.14,2.94,0,...,3895074,3.71,37.1,191.80,9,4.0,4.0,4.0,1.0,190.46
0,8481559,J. Hughes,NJD,C,21649,17,57.6,9.58,3.39,0,...,4565222,2.54,25.4,186.40,44,5.0,6.0,5.0,2.0,185.30
5,8477956,D. Pastrnak,BOS,R,26861,23,73.1,9.80,3.18,0,...,3114778,2.96,29.6,183.29,22,6.0,1.0,6.0,2.0,184.26
2,8479318,A. Matthews,TOR,C,28457,22,71.4,9.03,3.25,0,...,4024123,2.33,23.3,176.60,62,7.0,2.0,7.0,3.0,176.73
7,8480865,N. Dobson,NYI,D,34704,23,68.7,7.13,2.99,0,...,4352732,3.02,30.2,177.00,20,8.0,15.0,8.0,2.0,176.03
10,8477492,N. MacKinnon,COL,C,30923,23,67.3,7.83,2.93,0,...,3041969,3.06,30.6,175.43,19,9.0,8.0,9.0,4.0,174.27
12,8477939,W. Nylander,TOR,R,27297,22,62.4,8.23,2.84,0,...,3114736,2.97,29.7,173.00,21,10.0,12.0,10.0,3.0,173.12


In [7]:
# player_bios.to_excel('data/manual-adjustments.xlsx', index=False)

player_bios_adjusted = pd.read_excel('data/manual-adjustments.xlsx')

# vasil = {'playerId': 8476883, 'name': 'A. Vasilevskiy', 'team': 'TBL', 'position': 'G', 'default_pos': 'G', 'espnId': 2976847, 'remain_rank': 55, 'fullName': 'Andrei Vasilevskiy'}
# player_bios_adjusted = player_bios_adjusted.append(vasil, ignore_index=True)

kane = {'playerId': 8474141, 'name': 'P. Kane', 'team': 'DET', 'position': 'R', 'default_pos': 'RW', 'espnId': 3735, 'remain_rank': 125, 'fullName': 'Patrick Kane'}
player_bios_adjusted = player_bios_adjusted.append(kane, ignore_index=True)

# for index, row in player_bios_adjusted.iterrows():
#     player_bios_adjusted.at[index, 'fullName'] = player_bios.loc[player_bios['playerId'] == row['playerId'], 'fullName'].iloc[0]
#     player_bios_adjusted.at[index, 'default_pos'] = player_bios.loc[player_bios['playerId'] == row['playerId'], 'default_pos'].iloc[0]
    
player_bios_adjusted['espnId'] = player_bios_adjusted['espnId'].fillna(0).astype(int)
player_bios_adjusted = player_bios_adjusted.sort_values('remain_rank', ascending=True)
player_bios_adjusted['rank'] = player_bios_adjusted['remain_rank'].rank(ascending=True, method='first')
player_bios_adjusted['rank_by_pos'] = player_bios_adjusted.groupby('default_pos')['rank'].rank(ascending=True)
player_bios_adjusted.head(60)

player_bios_adjusted

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos,adjSkedFantasyRemain
0,8478402,C. McDavid,EDM,C,26057.0,20.0,58.9,8.14,2.94,0.0,...,3895074,3.71,37.1,191.80,9.0,1,4.0,1.0,1.0,190.46
1,8480069,C. Makar,COL,D,33757.0,23.0,75.9,8.09,3.30,0.0,...,4233563,3.69,36.9,202.37,10.0,2,5.0,2.0,1.0,202.18
2,8477956,D. Pastrnak,BOS,R,26861.0,23.0,73.1,9.80,3.18,0.0,...,3114778,2.96,29.6,183.29,22.0,3,1.0,3.0,1.0,184.26
3,8476453,N. Kucherov,TBL,R,31088.0,24.0,82.5,9.55,3.44,0.0,...,2563060,3.32,33.2,193.80,13.0,4,3.0,4.0,2.0,192.09
4,8479318,A. Matthews,TOR,C,28457.0,22.0,71.4,9.03,3.25,0.0,...,4024123,2.33,23.3,176.60,62.0,5,2.0,5.0,2.0,176.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,8470594,M. Fleury,MIN,G,32280.0,9.0,-4.6,-0.51,-0.51,0.0,...,2346,-0.51,-4.6,-12.75,822.0,826,818.0,827.0,72.0,-13.13
826,8476883,A. Vasilevskiy,TBL,G,13040.0,4.0,-7.0,-1.93,-1.75,0.0,...,2976847,-1.93,-7.0,-15.44,826.0,827,800.0,828.0,73.0,-15.61
827,8477480,E. Comrie,BUF,G,17144.0,6.0,-9.6,-2.02,-1.60,0.0,...,3042057,-2.02,-9.6,-22.22,828.0,828,814.0,829.0,74.0,-21.61
828,8475789,J. Campbell,EDM,G,16007.0,5.0,-8.6,-1.93,-1.72,0.0,...,5473,-1.93,-8.6,-23.16,827.0,829,820.0,830.0,75.0,-22.92


In [11]:
team_names_only = team_names[['abbreviation', 'name']]

espn_teams = {
    'BOS': 'Bos',
    'VGK': 'Vgk',
    'NYR': 'NYR',
    'VAN': 'Van',
    'LAK': 'LA',
    'DAL': 'Dal',
    'FLA': 'Fla',
    'COL': 'Col',
    'WPG': 'Wpg',
    'TOR': 'Tor',
    'WSH': 'Wsh',
    'CAR': 'Car',
    'TBL': 'TB',
    'PHI': 'Phi',
    'DET': 'Det',
    'ANA': 'Ana',
    'ARI': 'Ari',
    'SEA': 'Sea',
    'NJD': 'NJ',
    'STL': 'StL',
    'NYI': 'NYI',
    'OTT': 'Ott',
    'PIT': 'Pit',
    'MTL': 'Mon',
    'BUF': 'Buf',
    'CGY': 'Cgy',
    'MIN': 'Min',
    'NSH': 'Nsh',
    'CBJ': 'Cls',
    'EDM': 'Edm',
    'CHI': 'Chi',
    'SJS': 'SJ'
}

team_names_only['espn_name'] = team_names_only['abbreviation'].map(espn_teams)
team_names_only

,abbreviation,name,espn_name
0,NYR,New York Rangers,NYR
1,BOS,Boston Bruins,Bos
2,VGK,Vegas Golden Knights,Vgk
3,VAN,Vancouver Canucks,Van
4,COL,Colorado Avalanche,Col
5,LAK,Los Angeles Kings,LA
6,DAL,Dallas Stars,Dal
7,FLA,Florida Panthers,Fla
8,DET,Detroit Red Wings,Det
9,CAR,Carolina Hurricanes,Car


In [13]:
fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Benoit-Olivier Groulx': 'Bo Groulx',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
#     'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker',
    'Alex Barre-Boulet': 'Alex Barré-Boulet'
}

inverse_fixes = {value: key for key, value in fixes.items()}

player_bios_adjusted['fullName'].replace(inverse_fixes, inplace=True)

player_bios_adjusted

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,espnId,last10FPPG,last10FP,adjFantasyRemain,last10rank,remain_rank,last_rank,rank,rank_by_pos,adjSkedFantasyRemain
0,8478402,C. McDavid,EDM,C,26057.0,20.0,58.9,8.14,2.94,0.0,...,3895074,3.71,37.1,191.80,9.0,1,4.0,1.0,1.0,190.46
1,8480069,C. Makar,COL,D,33757.0,23.0,75.9,8.09,3.30,0.0,...,4233563,3.69,36.9,202.37,10.0,2,5.0,2.0,1.0,202.18
2,8477956,D. Pastrnak,BOS,R,26861.0,23.0,73.1,9.80,3.18,0.0,...,3114778,2.96,29.6,183.29,22.0,3,1.0,3.0,1.0,184.26
3,8476453,N. Kucherov,TBL,R,31088.0,24.0,82.5,9.55,3.44,0.0,...,2563060,3.32,33.2,193.80,13.0,4,3.0,4.0,2.0,192.09
4,8479318,A. Matthews,TOR,C,28457.0,22.0,71.4,9.03,3.25,0.0,...,4024123,2.33,23.3,176.60,62.0,5,2.0,5.0,2.0,176.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,8470594,M. Fleury,MIN,G,32280.0,9.0,-4.6,-0.51,-0.51,0.0,...,2346,-0.51,-4.6,-12.75,822.0,826,818.0,827.0,72.0,-13.13
826,8476883,A. Vasilevskiy,TBL,G,13040.0,4.0,-7.0,-1.93,-1.75,0.0,...,2976847,-1.93,-7.0,-15.44,826.0,827,800.0,828.0,73.0,-15.61
827,8477480,E. Comrie,BUF,G,17144.0,6.0,-9.6,-2.02,-1.60,0.0,...,3042057,-2.02,-9.6,-22.22,828.0,828,814.0,829.0,74.0,-21.61
828,8475789,J. Campbell,EDM,G,16007.0,5.0,-8.6,-1.93,-1.72,0.0,...,5473,-1.93,-8.6,-23.16,827.0,829,820.0,830.0,75.0,-22.92


In [14]:
player_bios_adjusted
poor_rank = player_bios_adjusted.loc[(player_bios_adjusted['rank'] >= 250) & (player_bios_adjusted['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['last_rank']}")

B. Coleman: 228.0
N. Niederreiter: 148.0
M. Necas: 218.0
M. Del Gaizo: 54.0
N. Pionk: 234.0
B. Pesce: 174.0
L. Hughes: 236.0
T. Brodie: 237.0
F. Zetterlund: 250.0
P. Laine: 223.0
M. Appleton: 186.0
D. Strome: 207.0
V. Tarasenko: 215.0
J. Kotkaniemi: 221.0
M. Lohrei: 54.0
D. Fabbro: 205.0
J. Farabee: 198.0
A. Carrier: 118.0
T. Luneau: 54.0
J. van Riemsdyk: 244.0
N. Paul: 191.0
R. Strome: 197.0
S. Monahan: 183.0
A. Iafallo: 233.0
J. Schwartz: 151.0
M. Pospisil: 54.0
T. Terry: 175.0
R. Smith: 229.0
D. Hunt: 54.0
H. Haman Aktell: 54.0
J. Ludvig: 54.0
E. Andrae: 54.0
J. Struble: 54.0
V. Husso: 235.0
R. Jarventie: 54.0
J. Sourdif: 54.0
M. Samoskevich: 54.0
L. Karlsson: 54.0
R. Johnson: 54.0
I. Solovyov: 54.0
N. Matinpalo: 54.0
L. Rousek: 54.0
H. Shepard: 54.0
R. Lavoie: 54.0
F. Minten: 54.0
M. Studenic: 54.0
R. Winterton: 54.0
J. Allen: 248.0
S. Malinski: 54.0
S. Laberge: 54.0
M. Lajoie: 54.0
O. Pavel: 54.0
I. Rosen: 54.0
M. Tomkins: 54.0
M. Savoie: 54.0
M. Chrona: 54.0


In [17]:
# 1. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C1) <br />

ranking_module = pd.DataFrame()

for index, row in player_bios_adjusted.iterrows():
    base_string = '<a href="http://www.espn.com/nhl/player/_/id/'
    rank = str(int(row['rank']))
    Pid = str(row['espnId']) + '/'
    name = row['fullName']
    slug = convert_to_slug(name)
    pos = row['default_pos']
    team = team_names_only.loc[team_names_only['abbreviation'] == row['team']]['espn_name'].iloc[0]
    pos_rank = pos + str(int(row['rank_by_pos']))
    
    assemble = rank + '. ' + base_string + Pid + slug + '">' + name + '</a>, ' + pos + ', ' + team + ' (' + pos_rank + ')'
    
    temp = pd.DataFrame({'column': [assemble]})
    
    ranking_module = pd.concat([ranking_module, temp])
    
    print(f"{assemble}")
    
    if int(rank) == 250:
        break


1. <a href="http://www.espn.com/nhl/player/_/id/3895074/connor-mcdavid">Connor McDavid</a>, C, Edm (C1)
2. <a href="http://www.espn.com/nhl/player/_/id/4233563/cale-makar">Cale Makar</a>, D, Col (D1)
3. <a href="http://www.espn.com/nhl/player/_/id/3114778/david-pastrnak">David Pastrnak</a>, RW, Bos (RW1)
4. <a href="http://www.espn.com/nhl/player/_/id/2563060/nikita-kucherov">Nikita Kucherov</a>, RW, TB (RW2)
5. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C2)
6. <a href="http://www.espn.com/nhl/player/_/id/4565222/jack-hughes">Jack Hughes</a>, C, NJ (C3)
7. <a href="http://www.espn.com/nhl/player/_/id/5734/cam-talbot">Cam Talbot</a>, G, LA (G1)
8. <a href="http://www.espn.com/nhl/player/_/id/3041969/nathan-mackinnon">Nathan MacKinnon</a>, C, Col (C4)
9. <a href="http://www.espn.com/nhl/player/_/id/4352732/noah-dobson">Noah Dobson</a>, D, NYI (D2)
10. <a href="http://www.espn.com/nhl/player/_/id/3114736/william-nylander">William Nyl

In [106]:
fileName = f"data/2024Rankings-07-{(today+timedelta(days=1)).strftime('%b%d')}.xlsx"
ranking_module.to_excel(fileName, index=False, header=False)

In [100]:
for i in range(250):
    print(f"{player_bios_adjusted.iloc[i, 1]}: {int(player_bios_adjusted.iloc[i, 26]) - int(player_bios_adjusted.iloc[i, 25])}")

C. McDavid: -3
C. Makar: -3
D. Pastrnak: 2
N. Kucherov: 1
A. Matthews: 3
J. Hughes: 0
C. Talbot: -13
N. MacKinnon: 0
N. Dobson: -6
W. Nylander: -2
J. Miller: -3
L. Draisaitl: 2
M. Rantanen: 4
F. Forsberg: -17
C. Hellebuyck: -51
E. Pettersson: 9
S. Reinhart: -12
B. Tkachuk: 1
J. Trouba: -8
K. Kaprizov: -1
A. Panarin: 10
E. Bouchard: -10
Q. Hughes: 10
J. Pavelski: -24
M. Seider: -11
J. Oettinger: 8
T. Demko: 5
A. Hill: 12
A. Fox: 1
K. Connor: 7
R. Dahlin: -4
J. Eriksson Ek: -5
Z. Hyman: -19
R. Josi: -39
M. Marner: 9
I. Shesterkin: 6
J. Eichel: -7
C. McAvoy: -1
S. Crosby: 20
I. Sorokin: -2
B. Point: 17
A. Kopitar: -14
T. Jarry: -54
J. Morrissey: -46
A. Kempe: -5
T. Stützle: 13
M. Matheson: -2
J. Swayman: -10
S. Stamkos: 11
J. Robertson: 4
A. Pietrangelo: 8
M. Tkachuk: 11
M. Rielly: -4
J. Sanderson: -8
J. Carlson: -28
A. DeBrincat: 1
A. Georgiev: 12
E. Kane: -54
C. Kreider: 25
M. Zibanejad: 7
B. Boeser: 10
M. Zuccarello: -13
V. Nichushkin: -67
B. Jenner: -15
T. Moore: -81
R. O'Reilly: 5
R.

ValueError: cannot convert float NaN to integer

In [18]:
fileName = f"data/rankings-final-{(today+timedelta(days=1)).strftime('%b%d')}.xlsx"
player_bios_adjusted.to_excel(fileName, index=False)